# Hybrid Monte Carlo

## Credit Risk Analysis

In this notebook we give an example of calculating Credit Valuation Adjustments (CVA). First we consider the case where credit risk is assumed independent of other market risk factors. In a second analysis we take into account correlation between credit spreads and market risk factors.

In [ ]:
import sys
sys.path.append('../')  # make python find our modules
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import QuantLib as ql

We use a Vanilla interest rate swap as example product. The product yields a list of payoffs per observation time.

The methodology can easily be adapted to other products. The products just need to implement the *cashFlows()* method.

In [ ]:
today     = ql.Date(5,ql.October,2020)
ql.Settings.instance().evaluationDate = today

In [ ]:
from hybmc.products.Swap import Swap
discYtsH   = ql.YieldTermStructureHandle(
                 ql.FlatForward(today,0.015,ql.Actual365Fixed()))
projYtsH   = ql.YieldTermStructureHandle(
                 ql.FlatForward(today,0.020,ql.Actual365Fixed()))
index      = ql.Euribor6M(projYtsH)
startDate  = ql.Date(12,ql.October,2020)
endDate    = ql.Date(12,ql.October,2030)
calendar   = ql.TARGET()
fixedTenor = ql.Period('1y')
floatTenor = ql.Period('6m')
fixedSchedule = ql.MakeSchedule(startDate,endDate,tenor=fixedTenor,calendar=calendar)
floatSchedule = ql.MakeSchedule(startDate,endDate,tenor=floatTenor,calendar=calendar)
couponDayCount = ql.Thirty360()
notional   = 1.0
fixedRate  = 0.02
fixedLeg   = ql.FixedRateLeg(fixedSchedule,couponDayCount,[notional],[fixedRate])
floatingLeg = ql.IborLeg([notional],floatSchedule,index)
#
swap = Swap([fixedLeg,floatingLeg],[1.0,-1.0],discYtsH)

In [ ]:
cfs = swap.cashFlows(0.0)
display([str(cf) for cf in cfs])

## Independent Credit Risk

In this subsection we analyse credit risk in the case that credit is independent of other market risk factors. This assumption leads to the classical CVA formula
$$
  CVA = \int_0^T \bar\lambda(s) e^{-\int_0^s \bar\lambda(u)du} \cdot
        \mathbb{E}\left[ e^{-\int_0^s r(u)du} (1-R) V(s)^+ \right] ds.
$$
In this setting, $\bar\lambda(s)$ is a hazard rate, $r(s)$ is the risk-free interest rate and $R$ is a recovery rate. The exposure term $V(s)^+$ is the risk-free price of the instrumnt at future time $s$ floored at zero.

We can re-write the CVA formula using a discrete set of observation times $0=t_0, t_1, \ldots, t_n=T$, survival probability $Q(t)=e^{-\int_0^t \bar\lambda(u)du}$ and numeraire $B(t)=e^{-\int_0^t r(u)du}$. Then
$$
  CVA = (1-R) \cdot \sum_{i=1}^n \left[ Q(t_{i-1}) - Q(t_{i}) \right] \cdot \frac{1}{2}
        \left[
          \mathbb{E}\left\{ \left[ \frac{V(t_{i-1})}{B(t_{i-1})} \right]^+ \right\} +
          \mathbb{E}\left\{ \left[ \frac{V(t_{i})}{B(t_{i})} \right]^+ \right\}
        \right].
$$

### How does this relate to hybrid modelling?

Discounted future payoffs $\frac{V(t_{i})}{B(t_{i})}$ are simulated within the hybrid Monte Carlo framework:

  1.  Define cash flows per observation time.
  2.  Derive a time line of cash flows.
  3.  Calculate scenario discounted cash flows.

For exposure simulation we just need to take the floored values
$$
  \left[ \frac{V(t_{i})}{B(t_{i})} \right]^+ =
  \max\left\{ \frac{V(t_{i})}{B(t_{i})}, 0 \right\}.
$$

Expectation \mathbb{E}\left\{ \cdot \right\} is approximated as usual via averging the simulated and flored samples.

The only remaining piece is calculating the weighted sum using survival probabilities $Q(t_{i})$ ($i=0,\ldots,n$).

### We start with exposure calculation

We need to specify the observation times.

In [ ]:
obsTimes = np.linspace(0.0,11.0,41)

Also we want to specify the number of paths and seed consistently.

In [ ]:
nPaths = 2**10
seed = 314159265359

Then we set up a model and a simulation.

In [ ]:
from hybmc.models.HullWhiteModel import HullWhiteModel
from hybmc.simulations.McSimulation import McSimulation
ratesModel = HullWhiteModel(discYtsH,0.03,np.array([10.0]),np.array([0.0050]))
mcsim = McSimulation(ratesModel,obsTimes,nPaths,seed,True)  # only few paths for demo

Calculate scenarios, take positive part and show exposure.

In [ ]:
V_samples = swap.scenarios(obsTimes,mcsim)
epeStandard = np.average(np.maximum(V_samples,0.0),axis=0)

In [ ]:
fig = px.line(x=obsTimes,y=epeStandard, labels={'x' : '$t$', 'y' : r'$\mathbb{E}\left\{ \left[ V(t)/B(t) \right]^+ \right\}$'})
fig.show()

### Specify survival probabilities

Credit curves are set up as yield term structures. Survival probabilities are calculated as (pseudo) discount factors.

In [ ]:
from hybmc.termstructures.YieldCurve import YieldCurve
spreadLevel = 0.05
spreadCurve = YieldCurve(spreadLevel)  # use 5% instantanous default probablility
Q = np.array([ spreadCurve.discount(t) for t in obsTimes ])
#
fig = px.line(x=obsTimes,y=Q, labels={'x' : '$t$', 'y' : '$Q(t)$'})
fig.show()

### Calculate CVA

We assume 40% recovery rate.

In [ ]:
R = 0.40
CVA = (1-R)/2 * np.sum([ (Q0 - Q1) * (V0 + V1)
    for Q0, Q1, V0, V1 in zip(Q[:-1],Q[1:],epeStandard[:-1],epeStandard[1:]) ])
display('CVA (Standard): %.4f' % CVA)

We find that the 10y ATM swap with a 5% counterparty credit spread and 40% recovery rate has a CVA of 37bp.

## Correlated Credit Risk

In this subsection we analyse the case of correlated credit and market risk. We follow the approach in Brigo/Vrins, Disentangling Wrong-Way Risk, 2016, Sec. 3.3 (https://ssrn.com/abstract=3366804).

The method is based on stochastic evolution of an intensity rate $\lambda(t)$ (Cox process setup). As a first test case we diffuse $\lambda(t)$ according to a Gaussian model and re-use the Hull White interest rate model.

### Spread model setup

We use the spread curve from independent credit risk simulation. Moreover, we assume 100bp hazard rate volatility and very low mean reversion.

In [ ]:
mean  = 0.0001  # 1bp
sigma = 0.0100
spreadModel = HullWhiteModel(spreadCurve,mean,np.array([10.0]),np.array([sigma]))

The Hull White model for spread has the disadvantage that simulated spreads can become negative. To mitigate that property we also set up an alternative model which lets vol go to zero if rates go to zero. 

In [ ]:
from hybmc.models.QuasiGaussianModel import QuasiGaussianModel
skew = 0.5*sigma/spreadLevel
qgModel = QuasiGaussianModel(spreadCurve,1,np.array([10.0]),np.array([[sigma]]),np.array([[skew]]),np.array([[-skew]]),np.array([0.01]),np.array([mean]),np.identity(1))
spreadModel = qgModel  #  switch override

In [ ]:
s = np.linspace(0.0,0.1,11)
X0 = qgModel.initialValues()
v = np.array([ qgModel.sigma_f(0.0,X0 + (spread - spreadLevel)) for spread in s ])[:,0]
fig = go.Figure()
fig.add_trace( go.Scatter(x=s, y=v, mode='lines', name='sigma_f') )
fig.update_layout(xaxis_title='$\lambda$', yaxis_title='volatility')
fig.show()

We want to analyse the impact of correlation on exposures and CVA. Thus we setup *three* hybrid credit-rates models:

  1.  Negative credit-rates correlation
  2.  Zero credit-rates correlation
  3.  Positive credit-rates correlation

In [ ]:
from hybmc.models.CreditModel import CreditModel
corrs = [ -0.85, 0.0, 0.85 ]
creditModels = []
for c in corrs:
    corr = np.eye(2)
    corr[0,1] = c
    corr[1,0] = c
    creditModels.append(CreditModel(ratesModel,['CP'],[spreadModel],corr))

### Model simulation

We simulate all the models.

In [ ]:
mcSims = []
for model in creditModels:
    mcSims.append(McSimulation(model,obsTimes,nPaths,seed,True))  # only few paths for demo

Since our rates model is the first component model in the credit hyprid model its simulation is not affected by correlation. Thus we only need to simulate it once (and not for each correlation).

In [ ]:
V_samples = swap.scenarios(obsTimes,mcSims[0])

However, we do need to simulate the exposure scaling factor and the scaled future swap scenario prices for each correlation.

In [ ]:
from hybmc.simulations.CreditPayoffs import zetaScenarios
zeta_samples = []
V_samples_scaled = []
for mcsim in mcSims:
    zeta_samples.append(zetaScenarios('CP',obsTimes,mcsim))
    V_samples_scaled.append(V_samples*zeta_samples[-1])

Now, exosure calculation is straight forward and analogous to standard CVA approach.

In [ ]:
epeWWR = []
for V_s in V_samples_scaled:
    epeWWR.append(np.average(np.maximum(V_s,0.0),axis=0))

Finally, we can compare standard CVA exposures and CVA exposures taking into account WWR and correlation.

In [ ]:
fig = go.Figure()
fig.add_trace( go.Scatter(x=obsTimes, y=epeStandard, mode='lines', name='CVA (Standard)') )
for c,epe in zip(corrs,epeWWR):
    fig.add_trace( go.Scatter(x=obsTimes, y=epe, mode='lines', name=r'CVA ($\rho=%.2f$)' % c ) )
fig.update_layout(xaxis_title='$t$', yaxis_title='Effective exposure')
fig.show()

Also, we can calculate the corresponding CVA numbers.

In [ ]:
R = 0.40
CVA = []
for c,epe in zip(corrs,epeWWR):
    CVA = (1-R)/2 * np.sum([ (Q0 - Q1) * (V0 + V1)
        for Q0, Q1, V0, V1 in zip(Q[:-1],Q[1:],epe[:-1],epe[1:]) ])
    display('CVA (rho=%.2f): %.4f' % (c,CVA))

From this analysis we see the followings points:

  1.  Non-independent stochastic credit spreads do impact CVA calculation
  2.  For a (fixed) receiver swap positive credit-rates correlation leads to smaller CVA (If spreads increase also rates increase, but the receiver swap PV decreases.)
  3.  Zero-correlation CVA is not equal to standard CVA assuming independent credit risk!